# Biostat 257 Homework 6

**This homework is optional. Do it if you want to get hands-on experience with derivation and implementation of MM algorithm. I am glad to answer any questions you have.**

Again we continue with the linear mixed effects model (LMM)
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effects predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effects predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

The log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ is 
$$
    \ell_i(\boldsymbol{\beta}, \mathbf{L}, \sigma_0^2) = - \frac{n_i}{2} \log (2\pi) - \frac{1}{2} \log \det \boldsymbol{\Omega}_i - \frac{1}{2} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y} - \mathbf{X}_i \boldsymbol{\beta}),
$$
where
$$
    \boldsymbol{\Omega}_i = \sigma^2 \mathbf{I}_{n_i} + \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T.
$$
Given $m$ independent data points $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$, $i=1,\ldots,m$, we seek the maximum likelihood estimate (MLE) by maximizing the log-likelihood
$$
\ell(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma_0^2) = \sum_{i=1}^m \ell_i(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma_0^2).
$$

In HW4 and HW5, we considered the nonlinear programming (NLP) and EM algorithm for optimization. In this assignment, we derive and implement an **minorization-maximization (MM) algorithm** for the same problem.

## Q1. (20 pts) Convex matrix functions

We say a matrix-valued function $f$ is (matrix) convex if
$$
f[\lambda \mathbf{A} + (1 - \lambda) \mathbf{B}] \preceq \lambda f(\mathbf{A}) + (1 - \lambda) f(\mathbf{B})
$$
for all $\mathbf{A}$, $\mathbf{B}$, and $\lambda \in (0, 1)$. 

1. Show that the matrix fractional function
$$
f(\mathbf{A}, \mathbf{B}) = \mathbf{A}^T \mathbf{B}^{-1} \mathbf{A}
$$
is jointly convex in $m \times n$ matrix $\mathbf{A}$ and $m \times m$ positive definite matrix $\mathbf{B}$. 

2. Show that the log determinant function
$$
f(\mathbf{B}) = \log \det \mathbf{B}
$$
is concave on the set of positive definite matrix.

## Q2. (20 pts) MM derivation - minorization step

Let the covariance of $i$-th datum be
$$
\boldsymbol{\Omega}_i = \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T + \sigma^2 \mathbf{I}_{n_i}.
$$
and $\boldsymbol{\Omega}_i^{(t)}$ be the covariance matrix evaluated at current parameter iterate $(\boldsymbol{\Sigma}^{(t)}, \sigma^{2(t)})$
$$
\boldsymbol{\Omega}_i^{(t)} = \mathbf{Z}_i \boldsymbol{\Sigma}^{(t)} \mathbf{Z}_i^T + \sigma^{2(t)} \mathbf{I}_{n_i}.
$$

1. From Q1.1, show that 
\begin{eqnarray*}
& & \boldsymbol{\Omega}_i^{(t)} \boldsymbol{\Omega}_i^{-1} \boldsymbol{\Omega}_i^{(t)} \\
&\preceq& \left( \mathbf{Z}_i \boldsymbol{\Sigma}^{(t)} \mathbf{Z}_i^T \right) \left( \mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T \right)^+ \left( \mathbf{Z}_i \boldsymbol{\Sigma}^{(t)} \mathbf{Z}_i^T \right) + \frac{\sigma^{4(t)}}{\sigma^2} \mathbf{I}_{n_i} \\
&=& \left( \mathbf{Z}_i \boldsymbol{\Sigma}^{(t)} \mathbf{Z}_i^T \mathbf{Z}_i^{T+} \right) \boldsymbol{\Sigma}^{-1} \left( \mathbf{Z}_i^+ \mathbf{Z}_i \boldsymbol{\Sigma}^{(t)} \mathbf{Z}_i^T \right) + \frac{\sigma^{4(t)}}{\sigma^2} \mathbf{I}_{n_i}.
\end{eqnarray*}
Thus
\begin{eqnarray*}
& & - \frac 12 (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-1} (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}) \\
&\succeq& - \frac 12 (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-(t)} \left( \mathbf{Z}_i \boldsymbol{\Sigma}^{(t)} \mathbf{Z}_i^T \mathbf{Z}_i^{T+} \right) \boldsymbol{\Sigma}^{-1} \left( \mathbf{Z}_i^+ \mathbf{Z}_i \boldsymbol{\Sigma}^{(t)} \mathbf{Z}_i^T \right) \boldsymbol{\Omega}_i^{-(t)} (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}) \\
& & - \frac{\sigma^{4(t)}}{\sigma^2} (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-2(t)} (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}).
\end{eqnarray*}

2. From Q1.2, show that
$$
\, - \frac 12 \log \det \boldsymbol{\Omega} \ge \, - \frac 12 \log \det \boldsymbol{\Omega}^{(t)} - \operatorname{tr} [\boldsymbol{\Omega}^{-(t)} (\boldsymbol{\Omega} - \boldsymbol{\Omega}^{(t)})].
$$
Hint: Support hyperplane inequality for convex function.

3. Combining 1 and 2, we obtain a minorization function
\begin{eqnarray*}
g(\boldsymbol{\Omega}, \sigma^2 \mid \boldsymbol{\Omega}^{(t)}, \sigma^{2(t)}) &=& \sum_i g_i(\boldsymbol{\Omega}, \sigma^2 \mid \boldsymbol{\Omega}^{(t)}, \sigma^{2(t)}) \\
    &=& - \frac 12 \sum_i \operatorname{tr} (\mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-(t)} \mathbf{Z}_i \boldsymbol{\Sigma}) - \frac 12 \sum_i \mathbf{r}_i^{(t)} \boldsymbol{\Sigma}^{-1} \mathbf{r}_i^{(t)} \\
    & & - \frac{\sigma^2}{2} \sum_i \operatorname{tr} \boldsymbol{\Omega}_i^{-(t)} - \frac{\sigma^{4(t)}}{2\sigma^2} \sum_i (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})^T \boldsymbol{\Omega}_i^{-2(t)} (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}) + c^{(t)}
\end{eqnarray*}
for the LMM log-likelihood, where $c^{(t)}$ is a constant irrelavent to optimization and
\begin{eqnarray*}
\mathbf{r}_i^{(t)} &=& \mathbf{Z}_i^+ \mathbf{Z}_i \boldsymbol{\Sigma}^{(t)} \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-(t)} (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}) \\
&=& \boldsymbol{\Sigma}^{(t)} \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-(t)} (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}).
\end{eqnarray*}

## Q3. (20 pts) MM derivation - maximization step

In the maximization step of the MM algorithm, we maximize the minorization function $g$. It turns out there are analytical update for the parameters $\boldsymbol{\beta}$, $\boldsymbol{\Sigma}$, and $\sigma^2$. 

1. Write down the analytical update of $\boldsymbol{\beta}$.

2. Write down the analytical update of $\sigma^2$. 

3. To update $\boldsymbol{\Sigma}$, we set the gradient to zero
$$
\frac{\partial}{\partial \boldsymbol{\Sigma}} g(\boldsymbol{\Omega}, \sigma^2 \mid \boldsymbol{\Omega}^{(t)}, \sigma^{2(t)}) = - \frac 12 \sum_i \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-(t)} \mathbf{Z}_i + \frac 12 \boldsymbol{\Sigma}^{-1} \left( \sum_i \mathbf{r}_i^{(t)} \mathbf{r}_i^{(t)T} \right) \boldsymbol{\Sigma}^{-1} = \mathbf{0}_{q \times q}.
$$
Find an analytical solution to the estimation equation
$$
\boldsymbol{\Sigma}^{-1} \left( \sum_i \mathbf{r}_i^{(t)} \mathbf{r}_i^{(t)T} \right) \boldsymbol{\Sigma}^{-1} = \sum_i \mathbf{Z}_i^T \boldsymbol{\Omega}_i^{-(t)} \mathbf{Z}_i.
$$

## Q4. (50 pts) Implementation

Mimic the code in HW4 and HW5 to implement the MM algorithm for finding the MLE of LMM model. 

1. Break complicated coding tasks into pieces: objective evaluator (10 pts), a single MM iteration (20 pts), a `fit` function for running MM iterations (10 pts). 

2. Modularize your code by small functions. Test the correctness and efficiency of each function separately. 

3. Test the MM algorithm on the same data (1000 individuals with 1500-2000 observations per individual). Make sure it achieves the same log-likelihood as EM or NLP solutions. (10 pts)

## Q5. (10 pts) MM vs EM vs Newton type algorithms

Using the same starting point and convergence criterion, contrast MM algorithm with the EM algorithm (HW5) and Newton type algorithms (HW4) in terms the convergence rate.

Keep in mind comparison of algorithms is very problem specific. Usually conclusion on one problem cannot be generalized to other problems. 